# Question2

## Data Read, Clean, and Summary

Below I read in the regression_data provided as a Pandas data frame and begin to clean the data before analysing the features we have to work with. Luckily, there is not much wrong with this data and it is already propely scaled to standard values across the features.

As for unnecessary data, upon analyzing the plot graph and correlation values below we can see that both red and green channels have a high corellation, but not enough to warrant removing one of them.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import random
import math
import sys

df = pd.read_table('classification_data.tsv')

#Converting string date time to datetime obj
#Testing for nan or empty values
nan = np.array(np.where(pd.isnull(df)))
empty = np.array(np.where(df.applymap(lambda x: x == '')))

corr = df.corr(method='pearson')
indices = np.where(corr > 0.90)
indices = [(corr.index[x], corr.columns[y]) for x, y in zip(*indices) if x != y and x < y]

#Number of features = number of columns - the target class field
features = len(df.columns) - 1

df['Class'] = df['Class'].apply(lambda val: int(val))

print("-----Data Summary-----")

print("Number of Columns: " + str(len(df.columns)))
print("Number of Valid Features: " + str(features))
print("Number of Entries: " + str(df.shape[0]))

if(nan):
    print("NaN Values Found: " + str(nan))
else:
    print("NaN Values Found: None")
    
if(empty):
    print("Empty Values Found: " + str(empty))
else:
    print("Empty Values Found: None")
    
if(len(indices) > 0):
    print("Correlated Values Found: " + str(indices))
else:
    print("Corellated Found: None")
    
print("\n-----General Dataframe Information-----\n")
print(df.describe())

def Plot(data, plot, columns, col_x,col_y):
    x = columns[col_x]
    y = columns[col_y]
    plot.set_xlabel(x)
    plot.set_ylabel(y)
    plot.set_title(x + " vs " + y)
    
    plot.scatter(data[x].values,data[y].values)

def Cross_Plot(data):
    cols = data.columns
    num_cols = features
    plt.figure(figsize=(200,150),facecolor='white')
    for col in range(0,num_cols):
            for cross_col in range(0,num_cols):
                if col != cross_col:
                    ax = plt.subplot2grid((num_cols, num_cols), (col, cross_col))
                    Plot(data, ax,cols,col, cross_col)
    plt.show()

#Running this takes some time, but a version of the output has been included in the mark down above
Cross_Plot(df.drop(['Class'],axis=1))

-----Data Summary-----
Number of Columns: 4
Number of Valid Features: 3
Number of Entries: 245057
NaN Values Found: None
Empty Values Found: None
Corellated Found: None

-----General Dataframe Information-----

                 Red          Green           Blue          Class
count  245057.000000  245057.000000  245057.000000  245057.000000
mean      125.065446     132.507327     123.177151       1.792461
std        62.255653      59.941197      72.562165       0.405546
min         0.000000       0.000000       0.000000       1.000000
25%        68.000000      87.000000      70.000000       2.000000
50%       139.000000     153.000000     128.000000       2.000000
75%       176.000000     177.000000     164.000000       2.000000
max       255.000000     255.000000     255.000000       2.000000


## Preparing the Data Sets

### Normalization

After we have read in and cleaned our data we can normalize all of our features and target values, but in this case we do not need to since all the features are on the same scale.

### Train / Test

Once our data has been fully prepared we can form a training and testing data set. We use our larger training set to form a hypothesis, or model, on how to best reach our training target values. We then apply our trained model agaisn't our test set in order to gauge our final accuracy.

### K-Fold Cross Validation

If we only use our training set as-is we may develop a case of overfitting towards the initial training set state. In order to guard agaisnt underfitting / overfitting in our average data set we need to apply some form of cross validation. K-Fold allows us to train over a series of folds with randomized training indexes. Each fold contains a validation set formed out of a different part of the full training set. This allows us random data entries for our training sets as well as full coverage for our validation sets.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing

def Get_KFolds(data, y_label, num_folds):
    #Creates 5 folds from the train/test set each with a separate training and test set
    folds = []
    kf = KFold(n_splits = num_folds)
    for train_index, test_index in kf.split(data):
        training = []
        test = []
        
        train_x = data.drop(y_label, axis=1).values
        train_y = data[y_label].values
        
        #Creates a training set within the fold
        x = []
        y = []
        id = []
        
        for index in train_index:
            x.append(train_x[index])
            y.append(train_y[index])
        training = [x,y]
        
        #Creates a test set within the fold
        x = []
        y = []
        id = []
        for index in test_index:
            x.append(train_x[index])
            y.append(train_y[index])
        test = [x,y]

        folds.append([training,test])
    
    return folds

seed(1)

train_df,test_df = train_test_split(df,test_size=0.2,random_state=42)
train_df = train_df[:5000]
test_df = test_df[:1000]

num_folds = 5
folds = Get_KFolds(train_df, 'Class', num_folds);
train = [train_df.drop(['Class'], axis = 1).values, train_df['Class'].values]
test = [test_df.drop(['Class'], axis = 1).values, test_df['Class'].values]


## Training and Testing¶

### Hyper Parameter Search

For some of our algorithms there are several parameters we can set that may have profound impacts on our model and in result our predictions. For example in a polynomial algorithm the degree defines wether the polynomial is squared, cubic, etc. In order to make an informed decision on our hyper parameters we can use tools like GridSearchCV. By providing GridSearchCV with our target algorithm and some target hyperparamters we can test all our options and choose the most appropriate parameter values.

In the instance below I seperated out the search so we can see the impacts of the search and training on each algorithm and kernel. I have also set k fold cross validation to 5 on each of the searches so we guarantee our analysis is appropriately distributed and is not overfitting our initial state.

### Training and Validation

Technically we can use the GridsearchCV object to run our 'best parameters' model, but in order to see our K-Fold validation and the r2/rmse scores for each model I've included an implementation. Essentially, we step through each fold and train our model using the hyperparamters we've identified and the training set of the fold. We then use the x values of our fold validation set to predict y values and compare them agaisn't our true y values. This will allow us to calculate aour variance and rmse scores. We do this for each fold and we can form our model while validating across our full distribution. The model that has performed the best on average during this validation period can then move on to final testing.

### Test Set

After we've selected the best hyperparameters and trained our models then we can start testing our hypothesis against untouched test data we've set aside. However, since this data is so large we need to sample the gridsearch on random data within the main set. Otherwise these searches take far too long. I've also discounted Kernel Ridge as possible method since the implementation often runs out of memory and is generealy unwieldy for this type of data.

### Result

Following these steps we can see that our results point to Soft SVM with a ploy kernel as the best model for this data. This makes a lot of sense since image implementations are one of SVMs key uses in Machine Learning.

In [ ]:
import gc
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix

#Grabbed this code from sklearn reference 
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

log_cv = GridSearchCV(estimator = linear_model.LogisticRegression(solver='sag'), cv=5, param_grid = {'max_iter':[4000],'C':[0.001,0.01,0.1,1,10,100,1000]}, n_jobs=-1)
log_cv.fit(train[0][:1000],train[1][:1000])
log_params = log_cv.best_params_

log = linear_model.LogisticRegression(solver='sag', max_iter = [4000])
log.set_params(**log_params)

print("Log Hyperparameters: {}".format(log_params))
sys.stdout.flush()

p_grid = [{'kernel':['linear'],'C':[0.1, 0.5, 1, 2, 5]}, 
         {'kernel':['poly'],'degree':[2,3],'C':[0.1, 0.5, 1, 2, 5, 1000]},
         {'kernel':['rbf'],'gamma':[0.1, 0.5, 1, 2, 5],'C':[0.1, 0.5, 1, 2, 5, 1000]}]

svm_cv = GridSearchCV(estimator = svm.SVC(), param_grid= p_grid, n_jobs=-1)
svm_cv.fit(train[0][:1000],train[1][:1000])
svm_params = svm_cv.best_params_

print("SVM Hyperparameters: {}".format(svm_params))
sys.stdout.flush()

svm_m = svm.SVC()
svm_m.set_params(**svm_params)

svm_avg = 0
log_avg = 0

for i in range(len(folds)):
    print('\n-----Fold #{}-----\n'.format(i + 1))
    x = folds[i][0][0]
    y = np.array(folds[i][0][1]).flatten()
    
    log.fit(x,y)
    log_score = log.score(folds[i][1][0],folds[i][1][1])
    log_avg += log_score / num_folds
    
    svm_m.fit(x,y)
    svm_score = svm_m.score(folds[i][1][0],folds[i][1][1])
    svm_avg += svm_score / num_folds
    
    print("\tLog Score: {}".format(log_score))
    print("\tSVM Score: {}".format(svm_score))
    sys.stdout.flush()
    
print("\n-----Training Average-----\n")
print("\tLog Score: {}".format(log_avg))
print("\tSVM Score: {}".format(svm_avg))
sys.stdout.flush()

print("\n-----Testing-----\n")
score = log.score(test[0],test[1])
predictions = log.predict(test[0])
log_matrix = confusion_matrix(test[1],predictions)
print("\tLog Score: {}".format(score))

score = svm_m.score(test[0],test[1])
predictions = svm_m.predict(test[0])
svm_matrix = confusion_matrix(test[1],predictions)
print("\tSVM Score: {}".format(score))
    
plot_confusion_matrix(log_matrix, classes=['1','2'],
                      title='Log Confusion matrix')

plt.show()

plot_confusion_matrix(svm_matrix, classes=['1','2'],
                      title='SVM Confusion matrix')

plt.show()

Log Hyperparameters: {'C': 0.1, 'max_iter': 4000}
SVM Hyperparameters: {'kernel': 'poly', 'C': 0.1, 'degree': 2}

-----Fold #1-----

	Log Score: 0.9183
	SVM Score: 0.9961

-----Fold #2-----

